# 🚀 AI-Driven Predictive Analytics for Business Decision Making

---

## Project Overview

This comprehensive notebook implements **two major AI-driven predictive analytics projects** designed to empower business decision-making:

### 📊 Project 1: Sales Forecasting Model
Predict future sales performance using **Random Forest**, **Gradient Boosting (XGBoost)**, and **LSTM Deep Learning** models based on historical data, customer behavior, and seasonal trends.

### 👥 Project 2: Customer Churn Prediction  
Predict customer churn using **Logistic Regression**, **Random Forest**, **XGBoost**, and **Deep Neural Networks** to help businesses retain customers by identifying at-risk clients.

---

**Skills:** Machine Learning, Deep Learning, Python, Data Analysis  
**Tools:** scikit-learn, XGBoost, LightGBM, TensorFlow/Keras, matplotlib, seaborn, plotly  
**Datasets:** Kaggle Sales Forecasting & Telco Customer Churn

## 1. Import Required Libraries and Setup

In [2]:
# ============================================================
# Import Required Libraries
# ============================================================
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Scikit-learn
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (mean_absolute_error, mean_squared_error, r2_score,
                             accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, roc_curve, precision_recall_curve,
                             confusion_matrix, classification_report, auc)
from sklearn.inspection import permutation_importance

# XGBoost & LightGBM
import xgboost as xgb
import lightgbm as lgb

# Imbalanced Learning
from imblearn.over_sampling import SMOTE

# TensorFlow/Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Reproducibility
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
tf.random.set_seed(RANDOM_STATE)

print("✅ All libraries imported successfully!")
print(f"NumPy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"TensorFlow: {tf.__version__}")
print(f"XGBoost: {xgb.__version__}")
print(f"Scikit-learn: {__import__('sklearn').__version__}")

✅ All libraries imported successfully!
NumPy: 2.4.2
Pandas: 3.0.0
TensorFlow: 2.20.0
XGBoost: 3.2.0
Scikit-learn: 1.8.0


## 2. Create Output Directory for Charts

In [3]:
# ============================================================
# Create Output Directories for Charts
# ============================================================
BASE_DIR = os.path.dirname(os.path.abspath('AI_Predictive_Analytics.ipynb'))
CHARTS_DIR = os.path.join(BASE_DIR, 'charts')

directories = [
    os.path.join(CHARTS_DIR, 'sales_forecasting'),
    os.path.join(CHARTS_DIR, 'customer_churn'),
    os.path.join(CHARTS_DIR, 'comparison'),
    os.path.join(CHARTS_DIR, 'dashboard'),
]

for d in directories:
    os.makedirs(d, exist_ok=True)

def save_chart(fig, filename, subfolder='', dpi=150):
    """Save a matplotlib figure to the charts directory."""
    if subfolder:
        path = os.path.join(CHARTS_DIR, subfolder, filename)
    else:
        path = os.path.join(CHARTS_DIR, filename)
    fig.savefig(path, dpi=dpi, bbox_inches='tight', facecolor='white', edgecolor='none')
    print(f"  💾 Saved: {path}")

def save_plotly_chart(fig, filename, subfolder=''):
    """Save a plotly figure as PNG."""
    if subfolder:
        path = os.path.join(CHARTS_DIR, subfolder, filename)
    else:
        path = os.path.join(CHARTS_DIR, filename)
    fig.write_image(path, scale=2)
    print(f"  💾 Saved: {path}")

print("✅ Chart directories created:")
for d in directories:
    print(f"   📁 {d}")

✅ Chart directories created:
   📁 /Users/usama/Desktop/dummy/charts/sales_forecasting
   📁 /Users/usama/Desktop/dummy/charts/customer_churn
   📁 /Users/usama/Desktop/dummy/charts/comparison
   📁 /Users/usama/Desktop/dummy/charts/dashboard


## 3. Download and Load Datasets

We use two real-world datasets:
- **Sales Data**: Superstore sales data with product categories, regions, and time-based features
- **Churn Data**: Telco Customer Churn dataset with customer demographics and service usage

In [4]:
# ============================================================
# Load Datasets
# ============================================================

# Load Sales Data
sales_df = pd.read_csv('sales_data.csv', encoding='latin-1')
print("=" * 60)
print("📊 SALES DATASET")
print("=" * 60)
print(f"Shape: {sales_df.shape}")
print(f"\nColumns: {list(sales_df.columns)}")
print(f"\nData Types:\n{sales_df.dtypes}")
print(f"\nFirst 5 rows:")
sales_df.head()

📊 SALES DATASET
Shape: (9800, 18)

Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales']

Data Types:
Row ID             int64
Order ID             str
Order Date           str
Ship Date            str
Ship Mode            str
Customer ID          str
Customer Name        str
Segment              str
Country              str
City                 str
State                str
Postal Code      float64
Region               str
Product ID           str
Category             str
Sub-Category         str
Product Name         str
Sales            float64
dtype: object

First 5 rows:


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0000,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0000,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0000,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0000,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0000,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680


In [5]:
# Load Churn Data
churn_df = pd.read_csv('churn_data.csv')
print("=" * 60)
print("👥 CUSTOMER CHURN DATASET")
print("=" * 60)
print(f"Shape: {churn_df.shape}")
print(f"\nColumns: {list(churn_df.columns)}")
print(f"\nData Types:\n{churn_df.dtypes}")
print(f"\nFirst 5 rows:")
churn_df.head()

👥 CUSTOMER CHURN DATASET
Shape: (7043, 21)

Columns: ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Data Types:
customerID              str
gender                  str
SeniorCitizen         int64
Partner                 str
Dependents              str
tenure                int64
PhoneService            str
MultipleLines           str
InternetService         str
OnlineSecurity          str
OnlineBackup            str
DeviceProtection        str
TechSupport             str
StreamingTV             str
StreamingMovies         str
Contract                str
PaperlessBilling        str
PaymentMethod           str
MonthlyCharges      float64
TotalCharges            str
Churn                   str
dtype: object

First 5

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.8500,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.9500,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.8500,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.3000,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.7000,151.65,Yes


In [6]:
# Dataset Summary Statistics
print("=" * 60)
print("📊 SALES DATA - Statistical Summary")
print("=" * 60)
print(sales_df.describe())
print(f"\nMissing Values:\n{sales_df.isnull().sum()}")
print(f"\nTotal Missing: {sales_df.isnull().sum().sum()}")
print("\n" + "=" * 60)
print("👥 CHURN DATA - Statistical Summary")
print("=" * 60)
print(churn_df.describe())
print(f"\nMissing Values:\n{churn_df.isnull().sum()}")
print(f"\nTotal Missing: {churn_df.isnull().sum().sum()}")

📊 SALES DATA - Statistical Summary
         Row ID  Postal Code      Sales
count 9800.0000    9789.0000  9800.0000
mean  4900.5000   55273.3224   230.7691
std   2829.1607   32041.2234   626.6519
min      1.0000    1040.0000     0.4440
25%   2450.7500   23223.0000    17.2480
50%   4900.5000   58103.0000    54.4900
75%   7350.2500   90008.0000   210.6050
max   9800.0000   99301.0000 22638.4800

Missing Values:
Row ID            0
Order ID          0
Order Date        0
Ship Date         0
Ship Mode         0
Customer ID       0
Customer Name     0
Segment           0
Country           0
City              0
State             0
Postal Code      11
Region            0
Product ID        0
Category          0
Sub-Category      0
Product Name      0
Sales             0
dtype: int64

Total Missing: 11

👥 CHURN DATA - Statistical Summary
       SeniorCitizen    tenure  MonthlyCharges
count      7043.0000 7043.0000       7043.0000
mean          0.1621   32.3711         64.7617
std           0.368

---
# PART 1: SALES FORECASTING MODEL
---
## 4. Exploratory Data Analysis - Sales Data

Comprehensive visual exploration of the sales dataset to understand patterns, distributions, and relationships.

In [7]:
# ============================================================
# 4.1 Sales Data Preparation for EDA
# ============================================================
sales_df['Order Date'] = pd.to_datetime(sales_df['Order Date'], format='%d/%m/%Y', dayfirst=True, errors='coerce')
# Try alternative format if parsing failed
if sales_df['Order Date'].isnull().sum() > len(sales_df) * 0.5:
    sales_df['Order Date'] = pd.to_datetime(sales_df['Order Date'], format='mixed', dayfirst=False)

sales_df['Ship Date'] = pd.to_datetime(sales_df['Ship Date'], format='%d/%m/%Y', dayfirst=True, errors='coerce')
if sales_df['Ship Date'].isnull().sum() > len(sales_df) * 0.5:
    sales_df['Ship Date'] = pd.to_datetime(sales_df['Ship Date'], format='mixed', dayfirst=False)

sales_df['Year'] = sales_df['Order Date'].dt.year
sales_df['Month'] = sales_df['Order Date'].dt.month
sales_df['Day'] = sales_df['Order Date'].dt.day
sales_df['DayOfWeek'] = sales_df['Order Date'].dt.dayofweek
sales_df['Quarter'] = sales_df['Order Date'].dt.quarter
sales_df['WeekOfYear'] = sales_df['Order Date'].dt.isocalendar().week.astype(int)
sales_df['IsWeekend'] = (sales_df['DayOfWeek'] >= 5).astype(int)
sales_df['Month_Name'] = sales_df['Order Date'].dt.month_name()

print("✅ Date features extracted")
print(f"Date range: {sales_df['Order Date'].min()} to {sales_df['Order Date'].max()}")
sales_df[['Order Date', 'Year', 'Month', 'Quarter', 'DayOfWeek', 'IsWeekend']].head(10)

✅ Date features extracted
Date range: 2015-01-03 00:00:00 to 2018-12-30 00:00:00


,Order Date,Year,Month,Quarter,DayOfWeek,IsWeekend
0,2017-11-08,2017,11,4,2,0
1,2017-11-08,2017,11,4,2,0
2,2017-06-12,2017,6,2,0,0
3,2016-10-11,2016,10,4,1,0
4,2016-10-11,2016,10,4,1,0
5,2015-06-09,2015,6,2,1,0
6,2015-06-09,2015,6,2,1,0
7,2015-06-09,2015,6,2,1,0
8,2015-06-09,2015,6,2,1,0
9,2015-06-09,2015,6,2,1,0


In [8]:
# ============================================================
# 4.2 Sales Distribution & Missing Values
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Sales Distribution
axes[0].hist(sales_df['Sales'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].set_title('Sales Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Sales ($)')
axes[0].set_ylabel('Frequency')
axes[0].axvline(sales_df['Sales'].mean(), color='red', linestyle='--', label=f"Mean: ${sales_df['Sales'].mean():.2f}")
axes[0].axvline(sales_df['Sales'].median(), color='green', linestyle='--', label=f"Median: ${sales_df['Sales'].median():.2f}")
axes[0].legend()

# Log-transformed Sales Distribution
axes[1].hist(np.log1p(sales_df['Sales']), bins=50, color='coral', edgecolor='black', alpha=0.7)
axes[1].set_title('Log-Transformed Sales Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Log(Sales)')
axes[1].set_ylabel('Frequency')

# Missing Values Heatmap
missing = sales_df.isnull().sum()
missing = missing[missing > 0]
if len(missing) > 0:
    axes[2].barh(missing.index, missing.values, color='tomato')
    axes[2].set_title('Missing Values Count', fontsize=14, fontweight='bold')
else:
    axes[2].text(0.5, 0.5, 'No Missing Values!', ha='center', va='center', fontsize=16, color='green', transform=axes[2].transAxes)
    axes[2].set_title('Missing Values Check', fontsize=14, fontweight='bold')

plt.tight_layout()
save_chart(fig, '01_sales_distribution.png', 'sales_forecasting')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/sales_forecasting/01_sales_distribution.png


In [9]:
# ============================================================
# 4.3 Sales Over Time - Time Series Plot
# ============================================================
daily_sales = sales_df.groupby('Order Date')['Sales'].sum().reset_index()
daily_sales = daily_sales.sort_values('Order Date')

fig, axes = plt.subplots(2, 1, figsize=(18, 10))

# Daily Sales
axes[0].plot(daily_sales['Order Date'], daily_sales['Sales'], color='steelblue', alpha=0.6, linewidth=0.8)
# Add 30-day rolling average
rolling_avg = daily_sales['Sales'].rolling(window=30, min_periods=1).mean()
axes[0].plot(daily_sales['Order Date'], rolling_avg, color='red', linewidth=2, label='30-Day Rolling Average')
axes[0].set_title('Daily Sales Over Time with Rolling Average', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Sales ($)')
axes[0].legend(fontsize=12)
axes[0].grid(True, alpha=0.3)

# Monthly Sales
monthly_sales = sales_df.groupby([sales_df['Order Date'].dt.to_period('M')])['Sales'].sum().reset_index()
monthly_sales['Order Date'] = monthly_sales['Order Date'].dt.to_timestamp()
axes[1].plot(monthly_sales['Order Date'], monthly_sales['Sales'], marker='o', color='darkgreen', linewidth=2, markersize=5)
axes[1].fill_between(monthly_sales['Order Date'], monthly_sales['Sales'], alpha=0.2, color='green')
axes[1].set_title('Monthly Sales Trend', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Total Sales ($)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
save_chart(fig, '02_sales_time_series.png', 'sales_forecasting')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/sales_forecasting/02_sales_time_series.png


In [10]:
# ============================================================
# 4.4 Seasonal & Category Analysis
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Monthly Seasonal Pattern
month_order = ['January','February','March','April','May','June','July','August','September','October','November','December']
monthly_pattern = sales_df.groupby('Month_Name')['Sales'].mean().reindex(month_order)
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, 12))
axes[0,0].bar(range(12), monthly_pattern.values, color=colors, edgecolor='black', alpha=0.8)
axes[0,0].set_xticks(range(12))
axes[0,0].set_xticklabels([m[:3] for m in month_order], rotation=45)
axes[0,0].set_title('Average Sales by Month (Seasonal Pattern)', fontsize=13, fontweight='bold')
axes[0,0].set_ylabel('Average Sales ($)')

# Sales by Category
cat_sales = sales_df.groupby('Category')['Sales'].agg(['sum', 'mean', 'count']).sort_values('sum', ascending=True)
axes[0,1].barh(cat_sales.index, cat_sales['sum'], color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black')
axes[0,1].set_title('Total Sales by Category', fontsize=13, fontweight='bold')
axes[0,1].set_xlabel('Total Sales ($)')
for i, v in enumerate(cat_sales['sum']):
    axes[0,1].text(v + 1000, i, f'${v:,.0f}', va='center', fontweight='bold')

# Sales by Region
region_sales = sales_df.groupby('Region')['Sales'].sum().sort_values(ascending=True)
axes[1,0].barh(region_sales.index, region_sales.values, color=['#FFB347', '#87CEEB', '#98D8C8', '#F7DC6F'], edgecolor='black')
axes[1,0].set_title('Total Sales by Region', fontsize=13, fontweight='bold')
axes[1,0].set_xlabel('Total Sales ($)')
for i, v in enumerate(region_sales.values):
    axes[1,0].text(v + 1000, i, f'${v:,.0f}', va='center', fontweight='bold')

# Sales by Segment
seg_sales = sales_df.groupby('Segment')['Sales'].sum().sort_values(ascending=True)
axes[1,1].barh(seg_sales.index, seg_sales.values, color=['#DDA0DD', '#90EE90', '#F0E68C'], edgecolor='black')
axes[1,1].set_title('Total Sales by Segment', fontsize=13, fontweight='bold')
axes[1,1].set_xlabel('Total Sales ($)')
for i, v in enumerate(seg_sales.values):
    axes[1,1].text(v + 1000, i, f'${v:,.0f}', va='center', fontweight='bold')

plt.tight_layout()
save_chart(fig, '03_sales_category_analysis.png', 'sales_forecasting')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/sales_forecasting/03_sales_category_analysis.png


In [11]:
# ============================================================
# 4.5 Sub-Category Analysis & Box Plots
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# Top 10 Sub-Categories by Sales
subcat_sales = sales_df.groupby('Sub-Category')['Sales'].sum().sort_values(ascending=True)
axes[0,0].barh(subcat_sales.index, subcat_sales.values, color=plt.cm.viridis(np.linspace(0.2, 0.9, len(subcat_sales))), edgecolor='black')
axes[0,0].set_title('Sales by Sub-Category', fontsize=13, fontweight='bold')
axes[0,0].set_xlabel('Total Sales ($)')

# Box Plot - Sales by Category
sales_df.boxplot(column='Sales', by='Category', ax=axes[0,1], patch_artist=True,
                 boxprops=dict(facecolor='lightblue'), medianprops=dict(color='red', linewidth=2))
axes[0,1].set_title('Sales Distribution by Category', fontsize=13, fontweight='bold')
axes[0,1].set_xlabel('Category')
axes[0,1].set_ylabel('Sales ($)')
plt.sca(axes[0,1])
plt.xticks(rotation=45)

# Quarterly Sales Trend
quarterly_sales = sales_df.groupby(['Year', 'Quarter'])['Sales'].sum().reset_index()
for year in quarterly_sales['Year'].unique():
    year_data = quarterly_sales[quarterly_sales['Year'] == year]
    axes[1,0].plot(year_data['Quarter'], year_data['Sales'], marker='o', linewidth=2, label=str(int(year)))
axes[1,0].set_title('Quarterly Sales by Year', fontsize=13, fontweight='bold')
axes[1,0].set_xlabel('Quarter')
axes[1,0].set_ylabel('Total Sales ($)')
axes[1,0].legend(title='Year')
axes[1,0].set_xticks([1, 2, 3, 4])

# Ship Mode Distribution
ship_counts = sales_df['Ship Mode'].value_counts()
axes[1,1].pie(ship_counts.values, labels=ship_counts.index, autopct='%1.1f%%', 
              colors=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'], startangle=90,
              explode=[0.05]*len(ship_counts))
axes[1,1].set_title('Ship Mode Distribution', fontsize=13, fontweight='bold')

plt.tight_layout()
save_chart(fig, '04_sales_subcategory_boxplot.png', 'sales_forecasting')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/sales_forecasting/04_sales_subcategory_boxplot.png


In [12]:
# ============================================================
# 4.6 Correlation Heatmap & Day-of-Week Analysis
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Correlation Heatmap for numerical features
num_cols = sales_df.select_dtypes(include=[np.number]).columns.tolist()
corr_matrix = sales_df[num_cols].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            ax=axes[0], square=True, linewidths=0.5, cbar_kws={'shrink': 0.8})
axes[0].set_title('Correlation Heatmap - Sales Features', fontsize=13, fontweight='bold')

# Sales by Day of Week
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_sales = sales_df.groupby('DayOfWeek')['Sales'].mean()
colors = ['#FF6B6B' if i >= 5 else '#4ECDC4' for i in range(7)]
axes[1].bar(range(7), dow_sales.values, color=colors, edgecolor='black', alpha=0.8)
axes[1].set_xticks(range(7))
axes[1].set_xticklabels([d[:3] for d in day_names], rotation=45)
axes[1].set_title('Average Sales by Day of Week', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Average Sales ($)')
axes[1].legend(['Weekend', 'Weekday'], loc='upper right')

plt.tight_layout()
save_chart(fig, '05_sales_correlation_dayofweek.png', 'sales_forecasting')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/sales_forecasting/05_sales_correlation_dayofweek.png


## 5. Exploratory Data Analysis - Customer Churn Data

Comprehensive visual exploration of the Telco Customer Churn dataset to understand churn patterns, demographics, and service usage.

In [13]:
# ============================================================
# 5.1 Churn Distribution & Demographics
# ============================================================
# Convert TotalCharges to numeric
churn_df['TotalCharges'] = pd.to_numeric(churn_df['TotalCharges'], errors='coerce')
churn_df['TotalCharges'].fillna(churn_df['TotalCharges'].median(), inplace=True)

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Churn Distribution - Pie Chart
churn_counts = churn_df['Churn'].value_counts()
axes[0,0].pie(churn_counts.values, labels=['No Churn', 'Churn'], autopct='%1.1f%%',
              colors=['#4ECDC4', '#FF6B6B'], startangle=90, explode=[0, 0.1],
              textprops={'fontsize': 14, 'fontweight': 'bold'})
axes[0,0].set_title('Customer Churn Distribution', fontsize=14, fontweight='bold')

# Churn Distribution - Count Plot
sns.countplot(data=churn_df, x='Churn', ax=axes[0,1], palette=['#4ECDC4', '#FF6B6B'], edgecolor='black')
axes[0,1].set_title('Churn Count', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('Count')
for p in axes[0,1].patches:
    axes[0,1].annotate(f'{int(p.get_height())}', (p.get_x() + p.get_width()/2., p.get_height()),
                       ha='center', va='bottom', fontsize=14, fontweight='bold')

# Gender vs Churn
ct_gender = pd.crosstab(churn_df['gender'], churn_df['Churn'], normalize='index') * 100
ct_gender.plot(kind='bar', ax=axes[1,0], color=['#4ECDC4', '#FF6B6B'], edgecolor='black', alpha=0.8)
axes[1,0].set_title('Churn Rate by Gender', fontsize=14, fontweight='bold')
axes[1,0].set_ylabel('Percentage (%)')
axes[1,0].set_xticklabels(axes[1,0].get_xticklabels(), rotation=0)
axes[1,0].legend(['No Churn', 'Churn'])

# Senior Citizen vs Churn
ct_senior = pd.crosstab(churn_df['SeniorCitizen'].map({0: 'Non-Senior', 1: 'Senior'}), 
                         churn_df['Churn'], normalize='index') * 100
ct_senior.plot(kind='bar', ax=axes[1,1], color=['#4ECDC4', '#FF6B6B'], edgecolor='black', alpha=0.8)
axes[1,1].set_title('Churn Rate by Senior Citizen Status', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('Percentage (%)')
axes[1,1].set_xticklabels(axes[1,1].get_xticklabels(), rotation=0)
axes[1,1].legend(['No Churn', 'Churn'])

plt.tight_layout()
save_chart(fig, '01_churn_distribution_demographics.png', 'customer_churn')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/customer_churn/01_churn_distribution_demographics.png


In [14]:
# ============================================================
# 5.2 Service Usage & Contract Analysis
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(18, 12))

# Internet Service vs Churn
ct_internet = pd.crosstab(churn_df['InternetService'], churn_df['Churn'], normalize='index') * 100
ct_internet.plot(kind='bar', ax=axes[0,0], color=['#4ECDC4', '#FF6B6B'], edgecolor='black', alpha=0.8)
axes[0,0].set_title('Churn Rate by Internet Service', fontsize=13, fontweight='bold')
axes[0,0].set_ylabel('Percentage (%)')
axes[0,0].set_xticklabels(axes[0,0].get_xticklabels(), rotation=0)
axes[0,0].legend(['No Churn', 'Churn'])

# Contract Type vs Churn
ct_contract = pd.crosstab(churn_df['Contract'], churn_df['Churn'], normalize='index') * 100
ct_contract.plot(kind='bar', ax=axes[0,1], color=['#4ECDC4', '#FF6B6B'], edgecolor='black', alpha=0.8)
axes[0,1].set_title('Churn Rate by Contract Type', fontsize=13, fontweight='bold')
axes[0,1].set_ylabel('Percentage (%)')
axes[0,1].set_xticklabels(axes[0,1].get_xticklabels(), rotation=0)
axes[0,1].legend(['No Churn', 'Churn'])

# Payment Method vs Churn
ct_payment = pd.crosstab(churn_df['PaymentMethod'], churn_df['Churn'], normalize='index') * 100
ct_payment.plot(kind='bar', ax=axes[1,0], color=['#4ECDC4', '#FF6B6B'], edgecolor='black', alpha=0.8)
axes[1,0].set_title('Churn Rate by Payment Method', fontsize=13, fontweight='bold')
axes[1,0].set_ylabel('Percentage (%)')
axes[1,0].set_xticklabels(axes[1,0].get_xticklabels(), rotation=30, ha='right')
axes[1,0].legend(['No Churn', 'Churn'])

# Partner & Dependents vs Churn
services = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
churn_rates = []
for s in services:
    rate = churn_df[churn_df[s] == 'Yes']['Churn'].value_counts(normalize=True).get('Yes', 0) * 100
    churn_rates.append(rate)
axes[1,1].barh(services, churn_rates, color=['#FFB347', '#87CEEB', '#98D8C8', '#DDA0DD'], edgecolor='black')
axes[1,1].set_title('Churn Rate by Service (Yes Only)', fontsize=13, fontweight='bold')
axes[1,1].set_xlabel('Churn Rate (%)')
for i, v in enumerate(churn_rates):
    axes[1,1].text(v + 0.5, i, f'{v:.1f}%', va='center', fontweight='bold')

plt.tight_layout()
save_chart(fig, '02_churn_service_contract.png', 'customer_churn')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/customer_churn/02_churn_service_contract.png


In [15]:
# ============================================================
# 5.3 Numerical Features Distribution & Correlation
# ============================================================
fig, axes = plt.subplots(2, 3, figsize=(20, 12))

# Tenure Distribution by Churn
for label, color in [('No', '#4ECDC4'), ('Yes', '#FF6B6B')]:
    subset = churn_df[churn_df['Churn'] == label]
    axes[0,0].hist(subset['tenure'], bins=30, alpha=0.6, color=color, label=f'Churn={label}', edgecolor='black')
axes[0,0].set_title('Tenure Distribution by Churn', fontsize=13, fontweight='bold')
axes[0,0].set_xlabel('Tenure (months)')
axes[0,0].legend()

# Monthly Charges Distribution by Churn
for label, color in [('No', '#4ECDC4'), ('Yes', '#FF6B6B')]:
    subset = churn_df[churn_df['Churn'] == label]
    axes[0,1].hist(subset['MonthlyCharges'], bins=30, alpha=0.6, color=color, label=f'Churn={label}', edgecolor='black')
axes[0,1].set_title('Monthly Charges by Churn', fontsize=13, fontweight='bold')
axes[0,1].set_xlabel('Monthly Charges ($)')
axes[0,1].legend()

# Total Charges Distribution by Churn
for label, color in [('No', '#4ECDC4'), ('Yes', '#FF6B6B')]:
    subset = churn_df[churn_df['Churn'] == label]
    axes[0,2].hist(subset['TotalCharges'], bins=30, alpha=0.6, color=color, label=f'Churn={label}', edgecolor='black')
axes[0,2].set_title('Total Charges by Churn', fontsize=13, fontweight='bold')
axes[0,2].set_xlabel('Total Charges ($)')
axes[0,2].legend()

# Box Plots - Tenure, MonthlyCharges, TotalCharges by Churn
for idx, col in enumerate(['tenure', 'MonthlyCharges', 'TotalCharges']):
    churn_df.boxplot(column=col, by='Churn', ax=axes[1, idx], patch_artist=True,
                     boxprops=dict(facecolor='lightblue'), medianprops=dict(color='red', linewidth=2))
    axes[1, idx].set_title(f'{col} by Churn Status', fontsize=13, fontweight='bold')
    axes[1, idx].set_xlabel('Churn')

plt.tight_layout()
save_chart(fig, '03_churn_numerical_distributions.png', 'customer_churn')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/customer_churn/03_churn_numerical_distributions.png


In [16]:
# ============================================================
# 5.4 Churn Correlation Heatmap & Stacked Bars
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Encode churn for correlation
churn_encoded = churn_df.copy()
churn_encoded['Churn_Binary'] = (churn_encoded['Churn'] == 'Yes').astype(int)
churn_num = churn_encoded.select_dtypes(include=[np.number])
corr = churn_num.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', center=0,
            ax=axes[0], square=True, linewidths=0.5, cbar_kws={'shrink': 0.8})
axes[0].set_title('Correlation Heatmap - Churn Features', fontsize=13, fontweight='bold')

# Stacked Bar - Multiple Services vs Churn
services_cat = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
churn_rates_services = []
for s in services_cat:
    if s in churn_df.columns:
        rate = churn_df[churn_df[s] == 'Yes']['Churn'].value_counts(normalize=True).get('Yes', 0) * 100
        no_rate = churn_df[churn_df[s] == 'No']['Churn'].value_counts(normalize=True).get('Yes', 0) * 100
        churn_rates_services.append({'Service': s, 'With Service': rate, 'Without Service': no_rate})

svc_df = pd.DataFrame(churn_rates_services)
x_pos = range(len(svc_df))
width = 0.35
axes[1].bar([p - width/2 for p in x_pos], svc_df['With Service'], width, label='With Service', color='#4ECDC4', edgecolor='black')
axes[1].bar([p + width/2 for p in x_pos], svc_df['Without Service'], width, label='Without Service', color='#FF6B6B', edgecolor='black')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels(svc_df['Service'], rotation=45, ha='right')
axes[1].set_title('Churn Rate: With vs Without Service', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Churn Rate (%)')
axes[1].legend()

plt.tight_layout()
save_chart(fig, '04_churn_correlation_services.png', 'customer_churn')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/customer_churn/04_churn_correlation_services.png


## 6. Data Preprocessing & Feature Engineering for Sales Forecasting

Prepare the sales data for ML models by aggregating daily sales, creating lag features, rolling statistics, and cyclical encodings.

In [17]:
# ============================================================
# 6. Sales Data - Preprocessing & Feature Engineering
# ============================================================

# Aggregate daily sales
daily_df = sales_df.groupby('Order Date').agg(
    Sales=('Sales', 'sum'),
    Quantity=('Sales', 'count')
).reset_index()
daily_df = daily_df.sort_values('Order Date').reset_index(drop=True)

# Fill missing dates with 0 sales
date_range = pd.date_range(start=daily_df['Order Date'].min(), end=daily_df['Order Date'].max())
daily_df = daily_df.set_index('Order Date').reindex(date_range, fill_value=0).reset_index()
daily_df.columns = ['Date', 'Sales', 'Quantity']

# Time features
daily_df['Year'] = daily_df['Date'].dt.year
daily_df['Month'] = daily_df['Date'].dt.month
daily_df['Day'] = daily_df['Date'].dt.day
daily_df['DayOfWeek'] = daily_df['Date'].dt.dayofweek
daily_df['Quarter'] = daily_df['Date'].dt.quarter
daily_df['WeekOfYear'] = daily_df['Date'].dt.isocalendar().week.astype(int)
daily_df['IsWeekend'] = (daily_df['DayOfWeek'] >= 5).astype(int)
daily_df['DayOfYear'] = daily_df['Date'].dt.dayofyear

# Cyclical encoding for month and day of week
daily_df['Month_sin'] = np.sin(2 * np.pi * daily_df['Month'] / 12)
daily_df['Month_cos'] = np.cos(2 * np.pi * daily_df['Month'] / 12)
daily_df['DayOfWeek_sin'] = np.sin(2 * np.pi * daily_df['DayOfWeek'] / 7)
daily_df['DayOfWeek_cos'] = np.cos(2 * np.pi * daily_df['DayOfWeek'] / 7)

# Lag features
for lag in [1, 3, 7, 14, 21, 30]:
    daily_df[f'Sales_lag_{lag}'] = daily_df['Sales'].shift(lag)

# Rolling window statistics
for window in [7, 14, 30]:
    daily_df[f'Sales_rolling_mean_{window}'] = daily_df['Sales'].rolling(window=window, min_periods=1).mean()
    daily_df[f'Sales_rolling_std_{window}'] = daily_df['Sales'].rolling(window=window, min_periods=1).std()
    daily_df[f'Sales_rolling_max_{window}'] = daily_df['Sales'].rolling(window=window, min_periods=1).max()
    daily_df[f'Sales_rolling_min_{window}'] = daily_df['Sales'].rolling(window=window, min_periods=1).min()

# Exponential moving averages
for span in [7, 14, 30]:
    daily_df[f'Sales_ema_{span}'] = daily_df['Sales'].ewm(span=span).mean()

# Drop rows with NaN (from lag features)
daily_df = daily_df.dropna().reset_index(drop=True)

print(f"✅ Feature engineering complete!")
print(f"Dataset shape: {daily_df.shape}")
print(f"Features: {list(daily_df.columns)}")
print(f"\nDate range: {daily_df['Date'].min()} to {daily_df['Date'].max()}")
daily_df.head(10)

✅ Feature engineering complete!
Dataset shape: (1428, 36)
Features: ['Date', 'Sales', 'Quantity', 'Year', 'Month', 'Day', 'DayOfWeek', 'Quarter', 'WeekOfYear', 'IsWeekend', 'DayOfYear', 'Month_sin', 'Month_cos', 'DayOfWeek_sin', 'DayOfWeek_cos', 'Sales_lag_1', 'Sales_lag_3', 'Sales_lag_7', 'Sales_lag_14', 'Sales_lag_21', 'Sales_lag_30', 'Sales_rolling_mean_7', 'Sales_rolling_std_7', 'Sales_rolling_max_7', 'Sales_rolling_min_7', 'Sales_rolling_mean_14', 'Sales_rolling_std_14', 'Sales_rolling_max_14', 'Sales_rolling_min_14', 'Sales_rolling_mean_30', 'Sales_rolling_std_30', 'Sales_rolling_max_30', 'Sales_rolling_min_30', 'Sales_ema_7', 'Sales_ema_14', 'Sales_ema_30']

Date range: 2015-02-02 00:00:00 to 2018-12-30 00:00:00


,Date,Sales,Quantity,Year,Month,Day,DayOfWeek,Quarter,WeekOfYear,IsWeekend,DayOfYear,Month_sin,Month_cos,DayOfWeek_sin,DayOfWeek_cos,Sales_lag_1,Sales_lag_3,Sales_lag_7,Sales_lag_14,Sales_lag_21,Sales_lag_30,Sales_rolling_mean_7,Sales_rolling_std_7,Sales_rolling_max_7,Sales_rolling_min_7,Sales_rolling_mean_14,Sales_rolling_std_14,Sales_rolling_max_14,Sales_rolling_min_14,Sales_rolling_mean_30,Sales_rolling_std_30,Sales_rolling_max_30,Sales_rolling_min_30,Sales_ema_7,Sales_ema_14,Sales_ema_30
0,2015-02-02,211.6460,3,2015,2,2,0,1,6,0,33,0.8660,0.5000,0.0000,1.0000,468.9000,240.5000,1097.2500,378.5940,0.0000,16.4480,234.6157,184.2730,468.9000,0.0000,389.5364,724.2957,2673.8700,0.0000,495.6602,1082.5475,4407.1000,0.0000,285.4799,325.3723,391.6922
1,2015-02-03,97.1120,2,2015,2,3,1,1,6,0,34,0.8660,0.5000,0.7818,0.6235,211.6460,290.6660,426.6700,2673.8700,3553.7950,288.0600,187.5360,168.4469,468.9000,0.0000,205.4823,305.4535,1097.2500,0.0000,489.2952,1084.3700,4407.1000,0.0000,238.3832,294.6220,370.1358
2,2015-02-04,134.3840,3,2015,2,4,2,1,6,0,35,0.8660,0.5000,0.9749,-0.2225,97.1120,468.9000,3.9280,0.0000,61.9600,19.5360,206.1726,151.0674,468.9000,0.0000,215.0811,300.5720,1097.2500,0.0000,493.1235,1082.8561,4407.1000,0.0000,212.3814,273.0652,353.0325
3,2015-02-05,0.0000,0,2015,2,5,3,1,6,0,36,0.8660,0.5000,0.4339,-0.9010,134.3840,211.6460,0.0000,0.0000,149.9500,4407.1000,206.1726,151.0674,468.9000,0.0000,215.0811,300.5720,1097.2500,0.0000,346.2202,793.9704,3553.7950,0.0000,159.2831,236.3737,327.6247
4,2015-02-06,330.5120,4,2015,2,6,4,1,6,0,37,0.8660,0.5000,-0.4339,-0.9010,0.0000,97.1120,240.5000,40.0800,299.9640,87.1580,219.0314,158.1416,468.9000,0.0000,235.8263,297.5722,1097.2500,0.0000,354.3320,792.4741,3553.7950,0.0000,202.0921,249.0099,327.8310
5,2015-02-07,180.3200,2,2015,2,7,5,1,6,1,38,0.8660,0.5000,-0.9749,-0.2225,330.5120,134.3840,290.6660,0.0000,0.0000,0.0000,203.2677,155.2848,468.9000,0.0000,248.7063,290.3955,1097.2500,0.0000,360.3426,790.3750,3553.7950,0.0000,196.6489,239.7979,317.3656
6,2015-02-08,14.5600,1,2015,2,8,6,1,6,1,39,0.8660,0.5000,-0.7818,0.6235,180.3200,0.0000,468.9000,0.0000,64.8640,40.5440,138.3620,115.6439,330.5120,0.0000,249.7463,289.4609,1097.2500,0.0000,359.4765,790.7517,3553.7950,0.0000,151.1256,209.6147,296.0198
7,2015-02-09,0.0000,0,2015,2,9,0,1,7,0,40,0.8660,0.5000,0.0000,1.0000,14.5600,330.5120,211.6460,1097.2500,378.5940,54.8300,108.1269,120.8411,330.5120,0.0000,171.3713,163.4608,468.9000,0.0000,357.6488,791.5430,3553.7950,0.0000,113.3435,181.5440,275.2764
8,2015-02-10,0.0000,0,2015,2,10,1,1,7,0,41,0.8660,0.5000,0.7818,0.6235,0.0000,180.3200,97.1120,426.6700,2673.8700,9.9400,94.2537,127.6964,330.5120,0.0000,140.8949,151.5408,468.9000,0.0000,357.3175,791.6957,3553.7950,0.0000,85.0073,157.2466,256.0932
9,2015-02-11,2043.4000,9,2015,2,11,2,1,7,0,42,0.8660,0.5000,0.9749,-0.2225,0.0000,14.5600,134.3840,3.9280,0.0000,0.0000,366.9703,749.9754,2043.4000,0.0000,286.5714,526.3958,2043.4000,0.0000,425.4308,845.9374,3553.7950,0.0000,574.6104,409.5579,380.0046


In [18]:
# ============================================================
# Train/Test Split (Temporal - no data leakage)
# ============================================================
feature_cols = [c for c in daily_df.columns if c not in ['Date', 'Sales']]
X = daily_df[feature_cols].values
y = daily_df['Sales'].values

# Use last 20% as test set (temporal split)
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]
dates_test = daily_df['Date'].values[split_idx:]

# Scale features
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

print(f"✅ Train/Test Split Complete")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features: {X_train.shape[1]}")
print(f"Train date range: {daily_df['Date'].iloc[0]} to {daily_df['Date'].iloc[split_idx-1]}")
print(f"Test date range: {daily_df['Date'].iloc[split_idx]} to {daily_df['Date'].iloc[-1]}")

✅ Train/Test Split Complete
Training set: 1142 samples
Test set: 286 samples
Features: 34
Train date range: 2015-02-02 00:00:00 to 2018-03-19 00:00:00
Test date range: 2018-03-20 00:00:00 to 2018-12-30 00:00:00


## 7. Sales Forecasting - Random Forest Model

In [19]:
# ============================================================
# 7. Random Forest Regressor for Sales Forecasting
# ============================================================
print("🌲 Training Random Forest Regressor...")

# Hyperparameter tuning
rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_model = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1)
rf_search = RandomizedSearchCV(rf_model, rf_params, n_iter=20, cv=3, scoring='neg_mean_absolute_error',
                                random_state=RANDOM_STATE, n_jobs=-1, verbose=0)
rf_search.fit(X_train_scaled, y_train)

rf_best = rf_search.best_estimator_
rf_pred = rf_best.predict(X_test_scaled)

# Metrics
rf_mae = mean_absolute_error(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_r2 = r2_score(y_test, rf_pred)
rf_mape = np.mean(np.abs((y_test - rf_pred) / (y_test + 1e-8))) * 100

print(f"\n✅ Random Forest - Best Parameters: {rf_search.best_params_}")
print(f"   MAE:  ${rf_mae:.2f}")
print(f"   RMSE: ${rf_rmse:.2f}")
print(f"   R²:   {rf_r2:.4f}")
print(f"   MAPE: {rf_mape:.2f}%")

🌲 Training Random Forest Regressor...

✅ Random Forest - Best Parameters: {'n_estimators': 300, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_depth': 20}
   MAE:  $971.37
   RMSE: $1547.21
   R²:   0.6228
   MAPE: 21246926644.18%


In [20]:
# ============================================================
# Random Forest - Visualization Charts
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# Actual vs Predicted
axes[0,0].plot(dates_test, y_test, label='Actual', color='steelblue', linewidth=1.5)
axes[0,0].plot(dates_test, rf_pred, label='Predicted', color='red', linewidth=1.5, alpha=0.8)
axes[0,0].set_title('Random Forest: Actual vs Predicted Sales', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Date')
axes[0,0].set_ylabel('Sales ($)')
axes[0,0].legend(fontsize=12)
axes[0,0].tick_params(axis='x', rotation=45)

# Residual Plot
residuals_rf = y_test - rf_pred
axes[0,1].scatter(rf_pred, residuals_rf, alpha=0.5, color='steelblue', s=20)
axes[0,1].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[0,1].set_title('Random Forest: Residual Plot', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Predicted Sales ($)')
axes[0,1].set_ylabel('Residuals ($)')

# Prediction Error Distribution
axes[1,0].hist(residuals_rf, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
axes[1,0].axvline(x=0, color='red', linestyle='--', linewidth=2)
axes[1,0].set_title('Random Forest: Prediction Error Distribution', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Prediction Error ($)')
axes[1,0].set_ylabel('Frequency')

# Feature Importance
importances = rf_best.feature_importances_
feat_imp = pd.Series(importances, index=feature_cols).sort_values(ascending=True)
feat_imp.tail(15).plot(kind='barh', ax=axes[1,1], color='steelblue', edgecolor='black')
axes[1,1].set_title('Random Forest: Top 15 Feature Importances', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Importance')

plt.tight_layout()
save_chart(fig, '06_rf_sales_results.png', 'sales_forecasting')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/sales_forecasting/06_rf_sales_results.png


## 8. Sales Forecasting - Gradient Boosting (XGBoost) Model

In [21]:
# ============================================================
# 8. XGBoost Gradient Boosting for Sales Forecasting
# ============================================================
print("🚀 Training XGBoost Regressor...")

# Split train into train/val for early stopping
X_tr, X_val, y_tr, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=RANDOM_STATE)

xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbosity=0
)

xgb_model.fit(
    X_tr, y_tr,
    eval_set=[(X_tr, y_tr), (X_val, y_val)],
    verbose=False
)

xgb_pred = xgb_model.predict(X_test_scaled)

# Metrics
xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
xgb_r2 = r2_score(y_test, xgb_pred)
xgb_mape = np.mean(np.abs((y_test - xgb_pred) / (y_test + 1e-8))) * 100

print(f"\n✅ XGBoost Results:")
print(f"   MAE:  ${xgb_mae:.2f}")
print(f"   RMSE: ${xgb_rmse:.2f}")
print(f"   R²:   {xgb_r2:.4f}")
print(f"   MAPE: {xgb_mape:.2f}%")

🚀 Training XGBoost Regressor...

✅ XGBoost Results:
   MAE:  $850.69
   RMSE: $1374.24
   R²:   0.7024
   MAPE: 97466863417.52%


In [22]:
# ============================================================
# XGBoost - Visualization Charts
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# Actual vs Predicted
axes[0,0].plot(dates_test, y_test, label='Actual', color='steelblue', linewidth=1.5)
axes[0,0].plot(dates_test, xgb_pred, label='Predicted (XGBoost)', color='orange', linewidth=1.5, alpha=0.8)
axes[0,0].set_title('XGBoost: Actual vs Predicted Sales', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Date')
axes[0,0].set_ylabel('Sales ($)')
axes[0,0].legend(fontsize=12)
axes[0,0].tick_params(axis='x', rotation=45)

# Learning Curves
evals_result = xgb_model.evals_result()
axes[0,1].plot(evals_result['validation_0']['rmse'], label='Train RMSE', color='steelblue')
axes[0,1].plot(evals_result['validation_1']['rmse'], label='Validation RMSE', color='orange')
axes[0,1].set_title('XGBoost: Learning Curves', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Boosting Rounds')
axes[0,1].set_ylabel('RMSE')
axes[0,1].legend(fontsize=12)

# Residual Analysis
residuals_xgb = y_test - xgb_pred
axes[1,0].scatter(xgb_pred, residuals_xgb, alpha=0.5, color='orange', s=20)
axes[1,0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1,0].set_title('XGBoost: Residual Plot', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Predicted Sales ($)')
axes[1,0].set_ylabel('Residuals ($)')

# Feature Importance (Gain-based)
xgb_imp = pd.Series(xgb_model.feature_importances_, index=feature_cols).sort_values(ascending=True)
xgb_imp.tail(15).plot(kind='barh', ax=axes[1,1], color='orange', edgecolor='black')
axes[1,1].set_title('XGBoost: Top 15 Feature Importances', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Importance')

plt.tight_layout()
save_chart(fig, '07_xgb_sales_results.png', 'sales_forecasting')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/sales_forecasting/07_xgb_sales_results.png


## 9. Sales Forecasting - LSTM Deep Learning Model

In [23]:
# ============================================================
# 9. LSTM Deep Learning for Sales Forecasting
# ============================================================
print("🧠 Training LSTM Neural Network...")

# Prepare sequence data
scaler_lstm = MinMaxScaler()
sales_values = daily_df['Sales'].values.reshape(-1, 1)
sales_scaled = scaler_lstm.fit_transform(sales_values)

LOOKBACK = 30

def create_sequences(data, lookback):
    X_seq, y_seq = [], []
    for i in range(lookback, len(data)):
        X_seq.append(data[i-lookback:i, 0])
        y_seq.append(data[i, 0])
    return np.array(X_seq), np.array(y_seq)

X_lstm, y_lstm = create_sequences(sales_scaled, LOOKBACK)
X_lstm = X_lstm.reshape((X_lstm.shape[0], X_lstm.shape[1], 1))

# Temporal split
split_lstm = int(len(X_lstm) * 0.8)
X_lstm_train, X_lstm_test = X_lstm[:split_lstm], X_lstm[split_lstm:]
y_lstm_train, y_lstm_test = y_lstm[:split_lstm], y_lstm[split_lstm:]

# Build LSTM Model
lstm_model = Sequential([
    Input(shape=(LOOKBACK, 1)),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)
])

lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=0)
]

history = lstm_model.fit(
    X_lstm_train, y_lstm_train,
    validation_data=(X_lstm_test, y_lstm_test),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=0
)

# Predict & inverse transform
lstm_pred_scaled = lstm_model.predict(X_lstm_test, verbose=0)
lstm_pred = scaler_lstm.inverse_transform(lstm_pred_scaled).flatten()
y_lstm_actual = scaler_lstm.inverse_transform(y_lstm_test.reshape(-1, 1)).flatten()

# Metrics
lstm_mae = mean_absolute_error(y_lstm_actual, lstm_pred)
lstm_rmse = np.sqrt(mean_squared_error(y_lstm_actual, lstm_pred))
lstm_r2 = r2_score(y_lstm_actual, lstm_pred)
lstm_mape = np.mean(np.abs((y_lstm_actual - lstm_pred) / (y_lstm_actual + 1e-8))) * 100

print(f"\n✅ LSTM Results:")
print(f"   MAE:  ${lstm_mae:.2f}")
print(f"   RMSE: ${lstm_rmse:.2f}")
print(f"   R²:   {lstm_r2:.4f}")
print(f"   MAPE: {lstm_mape:.2f}%")
print(f"\nModel Summary:")
lstm_model.summary()

🧠 Training LSTM Neural Network...

✅ LSTM Results:
   MAE:  $1681.48
   RMSE: $2377.96
   R²:   0.0378
   MAPE: 1380347717865.57%

Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 30, 64)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,573 (349.90 KB)

 Trainable params: 29,857 (116.63 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 59,716 (233.27 KB)

In [24]:
# ============================================================
# LSTM - Visualization Charts
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# Training/Validation Loss
axes[0,0].plot(history.history['loss'], label='Training Loss', color='steelblue', linewidth=2)
axes[0,0].plot(history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
axes[0,0].set_title('LSTM: Training & Validation Loss', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Epoch')
axes[0,0].set_ylabel('Loss (MSE)')
axes[0,0].legend(fontsize=12)

# Actual vs Predicted
axes[0,1].plot(y_lstm_actual, label='Actual', color='steelblue', linewidth=1.5)
axes[0,1].plot(lstm_pred, label='Predicted (LSTM)', color='green', linewidth=1.5, alpha=0.8)
axes[0,1].set_title('LSTM: Actual vs Predicted Sales', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Time Steps')
axes[0,1].set_ylabel('Sales ($)')
axes[0,1].legend(fontsize=12)

# Residual Plot
residuals_lstm = y_lstm_actual - lstm_pred
axes[1,0].scatter(range(len(residuals_lstm)), residuals_lstm, alpha=0.5, color='green', s=20)
axes[1,0].axhline(y=0, color='red', linestyle='--', linewidth=2)
axes[1,0].set_title('LSTM: Residual Plot', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Sample Index')
axes[1,0].set_ylabel('Residuals ($)')

# Training MAE
axes[1,1].plot(history.history['mae'], label='Training MAE', color='steelblue', linewidth=2)
axes[1,1].plot(history.history['val_mae'], label='Validation MAE', color='red', linewidth=2)
axes[1,1].set_title('LSTM: Training & Validation MAE', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Epoch')
axes[1,1].set_ylabel('MAE')
axes[1,1].legend(fontsize=12)

plt.tight_layout()
save_chart(fig, '08_lstm_sales_results.png', 'sales_forecasting')
plt.show()

  💾 Saved: /Users/usama/Desktop/dummy/charts/sales_forecasting/08_lstm_sales_results.png


## 10. Sales Forecasting - Model Comparison

In [25]:
# ============================================================
# 10. Sales Forecasting Model Comparison
# ============================================================
sales_comparison = pd.DataFrame({
    'Model': ['Random Forest', 'XGBoost', 'LSTM'],
    'MAE': [rf_mae, xgb_mae, lstm_mae],
    'RMSE': [rf_rmse, xgb_rmse, lstm_rmse],
    'R²': [rf_r2, xgb_r2, lstm_r2],
    'MAPE (%)': [rf_mape, xgb_mape, lstm_mape]
})

print("=" * 70)
print("📊 SALES FORECASTING - MODEL COMPARISON")
print("=" * 70)
print(sales_comparison.to_string(index=False))
print(f"\n🏆 Best Model: {sales_comparison.loc[sales_comparison['MAE'].idxmin(), 'Model']} (lowest MAE)")

# Comparison Charts
fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# Grouped Bar Chart - Metrics Comparison
metrics = ['MAE', 'RMSE']
x = np.arange(len(metrics))
width = 0.25
for i, model in enumerate(sales_comparison['Model']):
    values = [sales_comparison.loc[i, 'MAE'], sales_comparison.loc[i, 'RMSE']]
    axes[0,0].bar(x + i*width, values, width, label=model, edgecolor='black', alpha=0.8)
axes[0,0].set_xticks(x + width)
axes[0,0].set_xticklabels(metrics)
axes[0,0].set_title('Model Comparison: MAE & RMSE', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('Error ($)')
axes[0,0].legend()

# R² Score Comparison
colors = ['steelblue', 'orange', 'green']
axes[0,1].bar(sales_comparison['Model'], sales_comparison['R²'], color=colors, edgecolor='black', alpha=0.8)
axes[0,1].set_title('Model Comparison: R² Score', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('R² Score')
for i, v in enumerate(sales_comparison['R²']):
    axes[0,1].text(i, v + 0.01, f'{v:.4f}', ha='center', fontweight='bold')

# All Models Predictions Overlay (using RF & XGB test data)
axes[1,0].plot(dates_test, y_test, label='Actual', color='black', linewidth=2)
axes[1,0].plot(dates_test, rf_pred, label='Random Forest', color='steelblue', linewidth=1.5, alpha=0.7)
axes[1,0].plot(dates_test, xgb_pred, label='XGBoost', color='orange', linewidth=1.5, alpha=0.7)
axes[1,0].set_title('All Sales Models: Predictions Overlay', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Date')
axes[1,0].set_ylabel('Sales ($)')
axes[1,0].legend(fontsize=10)
axes[1,0].tick_params(axis='x', rotation=45)

# Radar Chart
categories = ['MAE\n(inverted)', 'RMSE\n(inverted)', 'R²', 'MAPE\n(inverted)']
fig_radar = plt.figure(figsize=(8, 8))
ax_radar = fig_radar.add_subplot(111, polar=True)
angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
angles += angles[:1]

for i, model in enumerate(sales_comparison['Model']):
    # Normalize: higher is better, so invert error metrics
    max_mae = sales_comparison['MAE'].max()
    max_rmse = sales_comparison['RMSE'].max()
    max_mape = sales_comparison['MAPE (%)'].max()
    values = [
        1 - sales_comparison.loc[i, 'MAE'] / (max_mae + 1e-8),
        1 - sales_comparison.loc[i, 'RMSE'] / (max_rmse + 1e-8),
        max(sales_comparison.loc[i, 'R²'], 0),
        1 - sales_comparison.loc[i, 'MAPE (%)'] / (max_mape + 1e-8)
    ]
    values += values[:1]
    ax_radar.plot(angles, values, 'o-', linewidth=2, label=model)
    ax_radar.fill(angles, values, alpha=0.1)

ax_radar.set_xticks(angles[:-1])
ax_radar.set_xticklabels(categories, fontsize=10)
ax_radar.set_title('Sales Models: Radar Comparison', fontsize=14, fontweight='bold', pad=20)
ax_radar.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
save_chart(fig_radar, '09_sales_radar_comparison.png', 'comparison')
plt.show()

# Box plot of prediction errors
fig_box, ax_box = plt.subplots(figsize=(10, 6))
errors_data = pd.DataFrame({
    'Random Forest': residuals_rf,
    'XGBoost': residuals_xgb[:len(residuals_rf)]
})
errors_data.boxplot(ax=ax_box, patch_artist=True,
                     boxprops=dict(facecolor='lightblue'), medianprops=dict(color='red', linewidth=2))
ax_box.set_title('Prediction Error Distribution by Model', fontsize=14, fontweight='bold')
ax_box.set_ylabel('Prediction Error ($)')
ax_box.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
save_chart(fig_box, '10_sales_error_boxplot.png', 'comparison')
plt.show()

save_chart(fig, '09_sales_model_comparison.png', 'comparison')
plt.show()

📊 SALES FORECASTING - MODEL COMPARISON
        Model       MAE      RMSE     R²           MAPE (%)
Random Forest  971.3678 1547.2128 0.6228   21246926644.1755
      XGBoost  850.6866 1374.2355 0.7024   97466863417.5179
         LSTM 1681.4763 2377.9608 0.0378 1380347717865.5696

🏆 Best Model: XGBoost (lowest MAE)
  💾 Saved: /Users/usama/Desktop/dummy/charts/comparison/09_sales_radar_comparison.png
  💾 Saved: /Users/usama/Desktop/dummy/charts/comparison/10_sales_error_boxplot.png
  💾 Saved: /Users/usama/Desktop/dummy/charts/comparison/09_sales_model_comparison.png


---
# PART 2: CUSTOMER CHURN PREDICTION
---
## 11. Data Preprocessing & Feature Engineering for Churn Prediction

Encode categorical variables, handle class imbalance with SMOTE, engineer new features, and prepare train/test splits.

In [26]:
# ============================================================
# 11. Churn Data - Preprocessing & Feature Engineering
# ============================================================

churn_processed = churn_df.copy()

# Drop customerID (not useful for prediction)
churn_processed = churn_processed.drop('customerID', axis=1)

# Convert TotalCharges to numeric (already done, but ensure)
churn_processed['TotalCharges'] = pd.to_numeric(churn_processed['TotalCharges'], errors='coerce')
churn_processed['TotalCharges'].fillna(churn_processed['TotalCharges'].median(), inplace=True)

# Feature Engineering
# Tenure groups
churn_processed['TenureGroup'] = pd.cut(churn_processed['tenure'], 
                                         bins=[0, 12, 24, 36, 48, 60, 72],
                                         labels=['0-12', '12-24', '24-36', '36-48', '48-60', '60-72'])

# Average monthly spend
churn_processed['AvgMonthlySpend'] = churn_processed['TotalCharges'] / (churn_processed['tenure'] + 1)

# Service count
service_cols = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
                'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
churn_processed['ServiceCount'] = 0
for col in service_cols:
    if col in churn_processed.columns:
        churn_processed['ServiceCount'] += (churn_processed[col].isin(['Yes', 'DSL', 'Fiber optic'])).astype(int)

# Contract value indicator
contract_map = {'Month-to-month': 0, 'One year': 1, 'Two year': 2}
churn_processed['ContractValue'] = churn_processed['Contract'].map(contract_map)

# Interaction feature
churn_processed['Tenure_x_Monthly'] = churn_processed['tenure'] * churn_processed['MonthlyCharges']

# Customer Lifetime Value estimate
churn_processed['CLV_Estimate'] = churn_processed['tenure'] * churn_processed['MonthlyCharges'] * 12

# Encode binary variables
binary_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling', 'Churn']
le = LabelEncoder()
for col in binary_cols:
    churn_processed[col] = le.fit_transform(churn_processed[col])

# One-hot encode multi-class categorical variables
multi_cat_cols = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                  'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
                  'Contract', 'PaymentMethod', 'TenureGroup']
churn_processed = pd.get_dummies(churn_processed, columns=multi_cat_cols, drop_first=True)

# Ensure all columns are numeric
for col in churn_processed.columns:
    if churn_processed[col].dtype == 'object':
        churn_processed[col] = le.fit_transform(churn_processed[col].astype(str))

print(f"✅ Churn data preprocessed!")
print(f"Shape: {churn_processed.shape}")
print(f"Target distribution:\n{churn_processed['Churn'].value_counts()}")
churn_processed.head()

✅ Churn data preprocessed!
Shape: (7043, 41)
Target distribution:
Churn
0    5174
1    1869
Name: count, dtype: int64


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,AvgMonthlySpend,ServiceCount,ContractValue,Tenure_x_Monthly,CLV_Estimate,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,TenureGroup_12-24,TenureGroup_24-36,TenureGroup_36-48,TenureGroup_48-60,TenureGroup_60-72
0,0,0,1,0,1,0,1,29.8500,29.8500,0,14.9250,2,0,29.8500,358.2000,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
1,1,0,0,0,34,1,0,56.9500,1889.5000,0,53.9857,4,1,1936.3000,23235.6000,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,True,False,False,False
2,1,0,0,0,2,1,1,53.8500,108.1500,1,36.0500,4,0,107.7000,1292.4000,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,1,0,0,0,45,0,0,42.3000,1840.7500,0,40.0163,4,1,1903.5000,22842.0000,True,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,True,False,False
4,0,0,0,0,2,1,1,70.7000,151.6500,1,50.5500,2,0,141.4000,1696.8000,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False


In [28]:
# ============================================================
# Train/Test Split with SMOTE for class imbalance
# ============================================================
X_churn = churn_processed.drop('Churn', axis=1)
y_churn = churn_processed['Churn']

# Store feature names
churn_feature_names = X_churn.columns.tolist()

# Stratified Train/Test Split
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X_churn, y_churn, test_size=0.2, random_state=RANDOM_STATE, stratify=y_churn
)

# Scale features
scaler_churn = StandardScaler()
X_train_c_scaled = scaler_churn.fit_transform(X_train_c)
X_test_c_scaled = scaler_churn.transform(X_test_c)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=RANDOM_STATE)
X_train_smote, y_train_smote = smote.fit_resample(X_train_c_scaled, y_train_c)

print(f"✅ Train/Test Split Complete")
print(f"Original Training set: {X_train_c.shape[0]} samples")
print(f"After SMOTE: {X_train_smote.shape[0]} samples")
print(f"Test set: {X_test_c.shape[0]} samples")
print(f"Features: {X_train_c.shape[1]}")
print(f"\nClass distribution after SMOTE:")
print(pd.Series(y_train_smote).value_counts())

ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

## 12. Customer Churn - Logistic Regression Baseline

In [ ]:
# ============================================================
# 12. Logistic Regression Baseline for Churn
# ============================================================
print("📊 Training Logistic Regression...")

lr_model = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=RANDOM_STATE)
lr_model.fit(X_train_smote, y_train_smote)

lr_pred = lr_model.predict(X_test_c_scaled)
lr_prob = lr_model.predict_proba(X_test_c_scaled)[:, 1]

# Metrics
lr_acc = accuracy_score(y_test_c, lr_pred)
lr_prec = precision_score(y_test_c, lr_pred)
lr_rec = recall_score(y_test_c, lr_pred)
lr_f1 = f1_score(y_test_c, lr_pred)
lr_auc = roc_auc_score(y_test_c, lr_prob)

print(f"\n✅ Logistic Regression Results:")
print(f"   Accuracy:  {lr_acc:.4f}")
print(f"   Precision: {lr_prec:.4f}")
print(f"   Recall:    {lr_rec:.4f}")
print(f"   F1-Score:  {lr_f1:.4f}")
print(f"   AUC-ROC:   {lr_auc:.4f}")
print(f"\nClassification Report:\n{classification_report(y_test_c, lr_pred)}")

In [ ]:
# ============================================================
# Logistic Regression - Visualization Charts
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Confusion Matrix
cm_lr = confusion_matrix(y_test_c, lr_pred)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0,0],
            xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
axes[0,0].set_title('Logistic Regression: Confusion Matrix', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Predicted')
axes[0,0].set_ylabel('Actual')

# ROC Curve
fpr_lr, tpr_lr, _ = roc_curve(y_test_c, lr_prob)
axes[0,1].plot(fpr_lr, tpr_lr, color='steelblue', linewidth=2, label=f'AUC = {lr_auc:.4f}')
axes[0,1].plot([0, 1], [0, 1], 'k--', linewidth=1)
axes[0,1].set_title('Logistic Regression: ROC Curve', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('False Positive Rate')
axes[0,1].set_ylabel('True Positive Rate')
axes[0,1].legend(fontsize=12)
axes[0,1].fill_between(fpr_lr, tpr_lr, alpha=0.1, color='steelblue')

# Precision-Recall Curve
prec_lr, rec_lr, _ = precision_recall_curve(y_test_c, lr_prob)
pr_auc_lr = auc(rec_lr, prec_lr)
axes[1,0].plot(rec_lr, prec_lr, color='steelblue', linewidth=2, label=f'PR AUC = {pr_auc_lr:.4f}')
axes[1,0].set_title('Logistic Regression: Precision-Recall Curve', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Recall')
axes[1,0].set_ylabel('Precision')
axes[1,0].legend(fontsize=12)

# Feature Coefficients
coefficients = pd.Series(np.abs(lr_model.coef_[0]), index=churn_feature_names).sort_values(ascending=True)
coefficients.tail(15).plot(kind='barh', ax=axes[1,1], color='steelblue', edgecolor='black')
axes[1,1].set_title('Logistic Regression: Top 15 Feature Importance (|coef|)', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('|Coefficient|')

plt.tight_layout()
save_chart(fig, '05_lr_churn_results.png', 'customer_churn')
plt.show()

## 13. Customer Churn - Random Forest Classifier

In [ ]:
# ============================================================
# 13. Random Forest Classifier for Churn
# ============================================================
print("🌲 Training Random Forest Classifier...")

rf_c_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_c_model = RandomForestClassifier(random_state=RANDOM_STATE, n_jobs=-1)
rf_c_search = RandomizedSearchCV(rf_c_model, rf_c_params, n_iter=20, cv=5, scoring='f1',
                                  random_state=RANDOM_STATE, n_jobs=-1, verbose=0)
rf_c_search.fit(X_train_smote, y_train_smote)

rf_c_best = rf_c_search.best_estimator_
rf_c_pred = rf_c_best.predict(X_test_c_scaled)
rf_c_prob = rf_c_best.predict_proba(X_test_c_scaled)[:, 1]

# Metrics
rf_c_acc = accuracy_score(y_test_c, rf_c_pred)
rf_c_prec = precision_score(y_test_c, rf_c_pred)
rf_c_rec = recall_score(y_test_c, rf_c_pred)
rf_c_f1 = f1_score(y_test_c, rf_c_pred)
rf_c_auc = roc_auc_score(y_test_c, rf_c_prob)

print(f"\n✅ Random Forest Classifier Results:")
print(f"   Best Params: {rf_c_search.best_params_}")
print(f"   Accuracy:  {rf_c_acc:.4f}")
print(f"   Precision: {rf_c_prec:.4f}")
print(f"   Recall:    {rf_c_rec:.4f}")
print(f"   F1-Score:  {rf_c_f1:.4f}")
print(f"   AUC-ROC:   {rf_c_auc:.4f}")
print(f"\nClassification Report:\n{classification_report(y_test_c, rf_c_pred)}")

In [ ]:
# ============================================================
# Random Forest Classifier - Visualization Charts
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Confusion Matrix
cm_rf_c = confusion_matrix(y_test_c, rf_c_pred)
sns.heatmap(cm_rf_c, annot=True, fmt='d', cmap='Greens', ax=axes[0,0],
            xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
axes[0,0].set_title('Random Forest: Confusion Matrix', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Predicted')
axes[0,0].set_ylabel('Actual')

# ROC Curve
fpr_rf_c, tpr_rf_c, _ = roc_curve(y_test_c, rf_c_prob)
axes[0,1].plot(fpr_rf_c, tpr_rf_c, color='green', linewidth=2, label=f'AUC = {rf_c_auc:.4f}')
axes[0,1].plot([0, 1], [0, 1], 'k--', linewidth=1)
axes[0,1].set_title('Random Forest: ROC Curve', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('False Positive Rate')
axes[0,1].set_ylabel('True Positive Rate')
axes[0,1].legend(fontsize=12)
axes[0,1].fill_between(fpr_rf_c, tpr_rf_c, alpha=0.1, color='green')

# Precision-Recall Curve
prec_rf_c, rec_rf_c, _ = precision_recall_curve(y_test_c, rf_c_prob)
pr_auc_rf_c = auc(rec_rf_c, prec_rf_c)
axes[1,0].plot(rec_rf_c, prec_rf_c, color='green', linewidth=2, label=f'PR AUC = {pr_auc_rf_c:.4f}')
axes[1,0].set_title('Random Forest: Precision-Recall Curve', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Recall')
axes[1,0].set_ylabel('Precision')
axes[1,0].legend(fontsize=12)

# Feature Importance
rf_c_imp = pd.Series(rf_c_best.feature_importances_, index=churn_feature_names).sort_values(ascending=True)
rf_c_imp.tail(15).plot(kind='barh', ax=axes[1,1], color='green', edgecolor='black')
axes[1,1].set_title('Random Forest: Top 15 Feature Importances', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Importance')

plt.tight_layout()
save_chart(fig, '06_rf_churn_results.png', 'customer_churn')
plt.show()

## 14. Customer Churn - XGBoost Classifier

In [ ]:
# ============================================================
# 14. XGBoost Classifier for Churn
# ============================================================
print("🚀 Training XGBoost Classifier...")

# Calculate scale_pos_weight
n_neg = (y_train_c == 0).sum()
n_pos = (y_train_c == 1).sum()
scale_pos = n_neg / n_pos

xgb_c_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    eval_metric='logloss',
    verbosity=0
)

# Split for early stopping
X_tr_c, X_val_c, y_tr_c, y_val_c = train_test_split(
    X_train_smote, y_train_smote, test_size=0.2, random_state=RANDOM_STATE
)

xgb_c_model.fit(
    X_tr_c, y_tr_c,
    eval_set=[(X_tr_c, y_tr_c), (X_val_c, y_val_c)],
    verbose=False
)

xgb_c_pred = xgb_c_model.predict(X_test_c_scaled)
xgb_c_prob = xgb_c_model.predict_proba(X_test_c_scaled)[:, 1]

# Metrics
xgb_c_acc = accuracy_score(y_test_c, xgb_c_pred)
xgb_c_prec = precision_score(y_test_c, xgb_c_pred)
xgb_c_rec = recall_score(y_test_c, xgb_c_pred)
xgb_c_f1 = f1_score(y_test_c, xgb_c_pred)
xgb_c_auc = roc_auc_score(y_test_c, xgb_c_prob)

print(f"\n✅ XGBoost Classifier Results:")
print(f"   Accuracy:  {xgb_c_acc:.4f}")
print(f"   Precision: {xgb_c_prec:.4f}")
print(f"   Recall:    {xgb_c_rec:.4f}")
print(f"   F1-Score:  {xgb_c_f1:.4f}")
print(f"   AUC-ROC:   {xgb_c_auc:.4f}")
print(f"\nClassification Report:\n{classification_report(y_test_c, xgb_c_pred)}")

In [ ]:
# ============================================================
# XGBoost Classifier - Visualization Charts
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Confusion Matrix
cm_xgb_c = confusion_matrix(y_test_c, xgb_c_pred)
sns.heatmap(cm_xgb_c, annot=True, fmt='d', cmap='Oranges', ax=axes[0,0],
            xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
axes[0,0].set_title('XGBoost: Confusion Matrix', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Predicted')
axes[0,0].set_ylabel('Actual')

# ROC Curve
fpr_xgb_c, tpr_xgb_c, _ = roc_curve(y_test_c, xgb_c_prob)
axes[0,1].plot(fpr_xgb_c, tpr_xgb_c, color='orange', linewidth=2, label=f'AUC = {xgb_c_auc:.4f}')
axes[0,1].plot([0, 1], [0, 1], 'k--', linewidth=1)
axes[0,1].set_title('XGBoost: ROC Curve', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('False Positive Rate')
axes[0,1].set_ylabel('True Positive Rate')
axes[0,1].legend(fontsize=12)
axes[0,1].fill_between(fpr_xgb_c, tpr_xgb_c, alpha=0.1, color='orange')

# Precision-Recall Curve
prec_xgb_c, rec_xgb_c, _ = precision_recall_curve(y_test_c, xgb_c_prob)
pr_auc_xgb_c = auc(rec_xgb_c, prec_xgb_c)
axes[1,0].plot(rec_xgb_c, prec_xgb_c, color='orange', linewidth=2, label=f'PR AUC = {pr_auc_xgb_c:.4f}')
axes[1,0].set_title('XGBoost: Precision-Recall Curve', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Recall')
axes[1,0].set_ylabel('Precision')
axes[1,0].legend(fontsize=12)

# Feature Importance (Gain-based)
xgb_c_imp = pd.Series(xgb_c_model.feature_importances_, index=churn_feature_names).sort_values(ascending=True)
xgb_c_imp.tail(15).plot(kind='barh', ax=axes[1,1], color='orange', edgecolor='black')
axes[1,1].set_title('XGBoost: Top 15 Feature Importances', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Importance')

plt.tight_layout()
save_chart(fig, '07_xgb_churn_results.png', 'customer_churn')
plt.show()

## 15. Customer Churn - Deep Neural Network

In [ ]:
# ============================================================
# 15. Deep Neural Network for Churn Prediction
# ============================================================
print("🧠 Training Deep Neural Network...")

n_features = X_train_smote.shape[1]

dnn_model = Sequential([
    Input(shape=(n_features,)),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

dnn_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Class weights
class_weights = {0: 1.0, 1: scale_pos}

dnn_callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=0)
]

dnn_history = dnn_model.fit(
    X_train_smote, y_train_smote,
    validation_data=(X_test_c_scaled, y_test_c),
    epochs=100,
    batch_size=64,
    callbacks=dnn_callbacks,
    class_weight=class_weights,
    verbose=0
)

# Predict with optimal threshold using ROC
dnn_prob = dnn_model.predict(X_test_c_scaled, verbose=0).flatten()
fpr_dnn, tpr_dnn, thresholds_dnn = roc_curve(y_test_c, dnn_prob)
optimal_idx = np.argmax(tpr_dnn - fpr_dnn)
optimal_threshold = thresholds_dnn[optimal_idx]
dnn_pred = (dnn_prob >= optimal_threshold).astype(int)

# Metrics
dnn_acc = accuracy_score(y_test_c, dnn_pred)
dnn_prec = precision_score(y_test_c, dnn_pred)
dnn_rec = recall_score(y_test_c, dnn_pred)
dnn_f1 = f1_score(y_test_c, dnn_pred)
dnn_auc = roc_auc_score(y_test_c, dnn_prob)

print(f"\n✅ Deep Neural Network Results:")
print(f"   Optimal Threshold: {optimal_threshold:.4f}")
print(f"   Accuracy:  {dnn_acc:.4f}")
print(f"   Precision: {dnn_prec:.4f}")
print(f"   Recall:    {dnn_rec:.4f}")
print(f"   F1-Score:  {dnn_f1:.4f}")
print(f"   AUC-ROC:   {dnn_auc:.4f}")
print(f"\nClassification Report:\n{classification_report(y_test_c, dnn_pred)}")
print(f"\nModel Summary:")
dnn_model.summary()

In [ ]:
# ============================================================
# Deep Neural Network - Visualization Charts
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Training/Validation Accuracy
axes[0,0].plot(dnn_history.history['accuracy'], label='Training Accuracy', color='steelblue', linewidth=2)
axes[0,0].plot(dnn_history.history['val_accuracy'], label='Validation Accuracy', color='red', linewidth=2)
axes[0,0].set_title('DNN: Training & Validation Accuracy', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Epoch')
axes[0,0].set_ylabel('Accuracy')
axes[0,0].legend(fontsize=12)

# Training/Validation Loss
axes[0,1].plot(dnn_history.history['loss'], label='Training Loss', color='steelblue', linewidth=2)
axes[0,1].plot(dnn_history.history['val_loss'], label='Validation Loss', color='red', linewidth=2)
axes[0,1].set_title('DNN: Training & Validation Loss', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Epoch')
axes[0,1].set_ylabel('Loss')
axes[0,1].legend(fontsize=12)

# Confusion Matrix
cm_dnn = confusion_matrix(y_test_c, dnn_pred)
sns.heatmap(cm_dnn, annot=True, fmt='d', cmap='Purples', ax=axes[1,0],
            xticklabels=['No Churn', 'Churn'], yticklabels=['No Churn', 'Churn'])
axes[1,0].set_title('DNN: Confusion Matrix', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Predicted')
axes[1,0].set_ylabel('Actual')

# ROC Curve
axes[1,1].plot(fpr_dnn, tpr_dnn, color='purple', linewidth=2, label=f'AUC = {dnn_auc:.4f}')
axes[1,1].plot([0, 1], [0, 1], 'k--', linewidth=1)
axes[1,1].scatter(fpr_dnn[optimal_idx], tpr_dnn[optimal_idx], color='red', s=100, zorder=5,
                  label=f'Optimal Threshold = {optimal_threshold:.4f}')
axes[1,1].set_title('DNN: ROC Curve with Optimal Threshold', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('False Positive Rate')
axes[1,1].set_ylabel('True Positive Rate')
axes[1,1].legend(fontsize=11)
axes[1,1].fill_between(fpr_dnn, tpr_dnn, alpha=0.1, color='purple')

plt.tight_layout()
save_chart(fig, '08_dnn_churn_results.png', 'customer_churn')
plt.show()

## 16. Churn Model Comparison & Evaluation

In [ ]:
# ============================================================
# 16. Churn Model Comparison
# ============================================================
churn_comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest', 'XGBoost', 'Deep Neural Network'],
    'Accuracy': [lr_acc, rf_c_acc, xgb_c_acc, dnn_acc],
    'Precision': [lr_prec, rf_c_prec, xgb_c_prec, dnn_prec],
    'Recall': [lr_rec, rf_c_rec, xgb_c_rec, dnn_rec],
    'F1-Score': [lr_f1, rf_c_f1, xgb_c_f1, dnn_f1],
    'AUC-ROC': [lr_auc, rf_c_auc, xgb_c_auc, dnn_auc]
})

print("=" * 80)
print("👥 CUSTOMER CHURN - MODEL COMPARISON")
print("=" * 80)
print(churn_comparison.to_string(index=False))
print(f"\n🏆 Best Model by F1: {churn_comparison.loc[churn_comparison['F1-Score'].idxmax(), 'Model']}")
print(f"🏆 Best Model by AUC: {churn_comparison.loc[churn_comparison['AUC-ROC'].idxmax(), 'Model']}")

In [ ]:
# ============================================================
# Churn Model Comparison - Visualization Charts
# ============================================================
fig, axes = plt.subplots(2, 2, figsize=(20, 14))

# Grouped Bar Chart - All Metrics
metrics_cols = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
x = np.arange(len(metrics_cols))
width = 0.2
colors_churn = ['steelblue', 'green', 'orange', 'purple']
for i, model in enumerate(churn_comparison['Model']):
    values = [churn_comparison.loc[i, m] for m in metrics_cols]
    axes[0,0].bar(x + i*width, values, width, label=model, color=colors_churn[i], edgecolor='black', alpha=0.8)
axes[0,0].set_xticks(x + 1.5*width)
axes[0,0].set_xticklabels(metrics_cols, rotation=15)
axes[0,0].set_title('Churn Models: All Metrics Comparison', fontsize=14, fontweight='bold')
axes[0,0].set_ylabel('Score')
axes[0,0].legend(fontsize=9, loc='lower right')
axes[0,0].set_ylim(0, 1.05)

# All ROC Curves Overlay
axes[0,1].plot(fpr_lr, tpr_lr, color='steelblue', linewidth=2, label=f'LR (AUC={lr_auc:.3f})')
axes[0,1].plot(fpr_rf_c, tpr_rf_c, color='green', linewidth=2, label=f'RF (AUC={rf_c_auc:.3f})')
axes[0,1].plot(fpr_xgb_c, tpr_xgb_c, color='orange', linewidth=2, label=f'XGB (AUC={xgb_c_auc:.3f})')
axes[0,1].plot(fpr_dnn, tpr_dnn, color='purple', linewidth=2, label=f'DNN (AUC={dnn_auc:.3f})')
axes[0,1].plot([0, 1], [0, 1], 'k--', linewidth=1)
axes[0,1].set_title('All Models: ROC Curves', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('False Positive Rate')
axes[0,1].set_ylabel('True Positive Rate')
axes[0,1].legend(fontsize=11)

# All Precision-Recall Curves
prec_dnn, rec_dnn, _ = precision_recall_curve(y_test_c, dnn_prob)
axes[1,0].plot(rec_lr, prec_lr, color='steelblue', linewidth=2, label=f'LR')
axes[1,0].plot(rec_rf_c, prec_rf_c, color='green', linewidth=2, label=f'RF')
axes[1,0].plot(rec_xgb_c, prec_xgb_c, color='orange', linewidth=2, label=f'XGB')
axes[1,0].plot(rec_dnn, prec_dnn, color='purple', linewidth=2, label=f'DNN')
axes[1,0].set_title('All Models: Precision-Recall Curves', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Recall')
axes[1,0].set_ylabel('Precision')
axes[1,0].legend(fontsize=11)

# Radar Chart for Churn Models
categories_c = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
angles_c = np.linspace(0, 2 * np.pi, len(categories_c), endpoint=False).tolist()
angles_c += angles_c[:1]

ax_radar_c = plt.subplot(2, 2, 4, polar=True)
for i, model in enumerate(churn_comparison['Model']):
    values = [churn_comparison.loc[i, m] for m in categories_c]
    values += values[:1]
    ax_radar_c.plot(angles_c, values, 'o-', linewidth=2, label=model, color=colors_churn[i])
    ax_radar_c.fill(angles_c, values, alpha=0.05, color=colors_churn[i])
ax_radar_c.set_xticks(angles_c[:-1])
ax_radar_c.set_xticklabels(categories_c, fontsize=9)
ax_radar_c.set_title('Churn Models: Radar Comparison', fontsize=14, fontweight='bold', pad=20)
ax_radar_c.legend(fontsize=8, loc='upper right', bbox_to_anchor=(1.4, 1.1))

plt.tight_layout()
save_chart(fig, '11_churn_model_comparison.png', 'comparison')
plt.show()

## 17. Business Insights & Recommendations Dashboard

Executive-level dashboards summarizing key findings for both projects.

In [ ]:
# ============================================================
# 17. Business Insights Dashboard - Sales Forecasting
# ============================================================
fig = plt.figure(figsize=(24, 16))
fig.suptitle('📊 SALES FORECASTING - EXECUTIVE DASHBOARD', fontsize=20, fontweight='bold', y=0.98)

# Panel 1: Sales Trend with Forecast
ax1 = fig.add_subplot(2, 3, 1)
ax1.plot(dates_test[-60:], y_test[-60:], label='Actual', color='steelblue', linewidth=2)
ax1.plot(dates_test[-60:], rf_pred[-60:], label='RF Forecast', color='red', linewidth=1.5, alpha=0.7)
ax1.plot(dates_test[-60:], xgb_pred[-60:], label='XGB Forecast', color='orange', linewidth=1.5, alpha=0.7)
ax1.fill_between(dates_test[-60:], 
                  np.minimum(rf_pred[-60:], xgb_pred[-60:]),
                  np.maximum(rf_pred[-60:], xgb_pred[-60:]), alpha=0.15, color='gray', label='Confidence Band')
ax1.set_title('Sales Forecast (Last 60 Days)', fontsize=12, fontweight='bold')
ax1.legend(fontsize=8)
ax1.tick_params(axis='x', rotation=45)

# Panel 2: Monthly Sales Pattern
ax2 = fig.add_subplot(2, 3, 2)
monthly_avg = sales_df.groupby('Month')['Sales'].mean()
colors_month = plt.cm.RdYlGn(np.linspace(0.2, 0.9, 12))
ax2.bar(range(1, 13), monthly_avg.values, color=colors_month, edgecolor='black')
ax2.set_title('Average Sales by Month', fontsize=12, fontweight='bold')
ax2.set_xlabel('Month')
ax2.set_ylabel('Avg Sales ($)')
ax2.set_xticks(range(1, 13))
peak_month = monthly_avg.idxmax()
ax2.annotate(f'Peak: Month {peak_month}', xy=(peak_month, monthly_avg.max()),
             xytext=(peak_month+1, monthly_avg.max()*1.1),
             arrowprops=dict(arrowstyle='->', color='red'), fontsize=10, color='red', fontweight='bold')

# Panel 3: Category Revenue
ax3 = fig.add_subplot(2, 3, 3)
cat_sales = sales_df.groupby('Category')['Sales'].sum().sort_values()
ax3.barh(cat_sales.index, cat_sales.values, color=['#FF6B6B', '#4ECDC4', '#45B7D1'], edgecolor='black')
ax3.set_title('Total Revenue by Category', fontsize=12, fontweight='bold')
ax3.set_xlabel('Total Sales ($)')
for i, v in enumerate(cat_sales.values):
    ax3.text(v + 100, i, f'${v:,.0f}', va='center', fontweight='bold', fontsize=10)

# Panel 4: Regional Sales
ax4 = fig.add_subplot(2, 3, 4)
region_sales = sales_df.groupby('Region')['Sales'].sum().sort_values()
ax4.pie(region_sales.values, labels=region_sales.index, autopct='%1.1f%%',
        colors=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'], startangle=90,
        textprops={'fontsize': 11})
ax4.set_title('Sales Distribution by Region', fontsize=12, fontweight='bold')

# Panel 5: Model Performance Summary
ax5 = fig.add_subplot(2, 3, 5)
ax5.axis('off')
table_data = []
for _, row in sales_comparison.iterrows():
    table_data.append([row['Model'], f"${row['MAE']:.0f}", f"${row['RMSE']:.0f}", f"{row['R²']:.3f}"])
table = ax5.table(cellText=table_data, colLabels=['Model', 'MAE', 'RMSE', 'R²'],
                   loc='center', cellLoc='center')
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1.2, 1.8)
for (i, j), cell in table.get_celld().items():
    if i == 0:
        cell.set_facecolor('#4ECDC4')
        cell.set_text_props(color='white', fontweight='bold')
ax5.set_title('Sales Model Performance Summary', fontsize=12, fontweight='bold')

# Panel 6: Key Metrics KPIs
ax6 = fig.add_subplot(2, 3, 6)
ax6.axis('off')
total_sales = sales_df['Sales'].sum()
avg_order = sales_df['Sales'].mean()
total_orders = len(sales_df)
best_model = sales_comparison.loc[sales_comparison['R²'].idxmax(), 'Model']
kpi_text = f"""
📈 KEY BUSINESS METRICS

Total Revenue: ${total_sales:,.0f}
Average Order Value: ${avg_order:,.2f}
Total Orders: {total_orders:,}
Best Forecasting Model: {best_model}
Best R² Score: {sales_comparison['R²'].max():.4f}
Forecast MAPE: {sales_comparison['MAPE (%)'].min():.1f}%
Peak Sales Month: {peak_month}
"""
ax6.text(0.1, 0.5, kpi_text, transform=ax6.transAxes, fontsize=13,
         verticalalignment='center', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
save_chart(fig, '01_sales_executive_dashboard.png', 'dashboard')
plt.show()

In [ ]:
# ============================================================
# 17b. Business Insights Dashboard - Customer Churn
# ============================================================
fig = plt.figure(figsize=(24, 16))
fig.suptitle('👥 CUSTOMER CHURN PREDICTION - EXECUTIVE DASHBOARD', fontsize=20, fontweight='bold', y=0.98)

# Panel 1: Churn Risk Distribution
ax1 = fig.add_subplot(2, 3, 1)
# Use the best model's probabilities for risk segmentation
best_prob = xgb_c_prob  # XGBoost typically best
risk_labels = pd.cut(best_prob, bins=[0, 0.3, 0.6, 1.0], labels=['Low Risk', 'Medium Risk', 'High Risk'])
risk_counts = risk_labels.value_counts()
ax1.pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%',
        colors=['#4ECDC4', '#FFB347', '#FF6B6B'], startangle=90, explode=[0, 0.05, 0.1],
        textprops={'fontsize': 12, 'fontweight': 'bold'})
ax1.set_title('Customer Risk Segmentation', fontsize=13, fontweight='bold')

# Panel 2: Churn Probability Distribution
ax2 = fig.add_subplot(2, 3, 2)
ax2.hist(best_prob, bins=30, color='steelblue', edgecolor='black', alpha=0.7, label='All Customers')
ax2.axvline(x=0.5, color='red', linestyle='--', linewidth=2, label='Default Threshold')
ax2.axvline(x=optimal_threshold, color='green', linestyle='--', linewidth=2, label=f'Optimal ({optimal_threshold:.2f})')
ax2.set_title('Churn Probability Distribution', fontsize=13, fontweight='bold')
ax2.set_xlabel('Churn Probability')
ax2.set_ylabel('Count')
ax2.legend(fontsize=10)

# Panel 3: Revenue at Risk
ax3 = fig.add_subplot(2, 3, 3)
churn_test_df = churn_df.iloc[X_test_c.index].copy()
churn_test_df['ChurnProb'] = best_prob
churn_test_df['RiskLevel'] = risk_labels.values
revenue_at_risk = churn_test_df.groupby('RiskLevel')['MonthlyCharges'].sum()
ax3.bar(revenue_at_risk.index, revenue_at_risk.values, color=['#4ECDC4', '#FFB347', '#FF6B6B'], edgecolor='black')
ax3.set_title('Monthly Revenue at Risk by Segment', fontsize=13, fontweight='bold')
ax3.set_ylabel('Monthly Revenue ($)')
for i, v in enumerate(revenue_at_risk.values):
    ax3.text(i, v + 50, f'${v:,.0f}', ha='center', fontweight='bold', fontsize=11)

# Panel 4: Top Churn Factors
ax4 = fig.add_subplot(2, 3, 4)
top_features = xgb_c_imp.tail(10)
ax4.barh(top_features.index, top_features.values, color=plt.cm.RdYlGn_r(np.linspace(0.2, 0.9, 10)), edgecolor='black')
ax4.set_title('Top 10 Churn Drivers', fontsize=13, fontweight='bold')
ax4.set_xlabel('Feature Importance')

# Panel 5: Model Performance Summary
ax5 = fig.add_subplot(2, 3, 5)
ax5.axis('off')
table_data_c = []
for _, row in churn_comparison.iterrows():
    table_data_c.append([row['Model'], f"{row['Accuracy']:.3f}", f"{row['F1-Score']:.3f}", f"{row['AUC-ROC']:.3f}"])
table_c = ax5.table(cellText=table_data_c, colLabels=['Model', 'Accuracy', 'F1', 'AUC'],
                     loc='center', cellLoc='center')
table_c.auto_set_font_size(False)
table_c.set_fontsize(11)
table_c.scale(1.2, 1.8)
for (i, j), cell in table_c.get_celld().items():
    if i == 0:
        cell.set_facecolor('#FF6B6B')
        cell.set_text_props(color='white', fontweight='bold')
ax5.set_title('Churn Model Performance Summary', fontsize=13, fontweight='bold')

# Panel 6: Key Metrics KPIs
ax6 = fig.add_subplot(2, 3, 6)
ax6.axis('off')
total_customers = len(churn_df)
churn_rate = (churn_df['Churn'] == 'Yes').mean() * 100
high_risk_count = (risk_labels == 'High Risk').sum()
best_churn_model = churn_comparison.loc[churn_comparison['AUC-ROC'].idxmax(), 'Model']
avg_monthly = churn_df['MonthlyCharges'].mean()
kpi_text_c = f"""
👥 KEY CHURN METRICS

Total Customers: {total_customers:,}
Overall Churn Rate: {churn_rate:.1f}%
High-Risk Customers (Test): {high_risk_count}
Best Model: {best_churn_model}
Best AUC-ROC: {churn_comparison['AUC-ROC'].max():.4f}
Best F1-Score: {churn_comparison['F1-Score'].max():.4f}
Avg Monthly Revenue/Customer: ${avg_monthly:.2f}
"""
ax6.text(0.1, 0.5, kpi_text_c, transform=ax6.transAxes, fontsize=13,
         verticalalignment='center', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='mistyrose', alpha=0.8))

plt.tight_layout()
save_chart(fig, '02_churn_executive_dashboard.png', 'dashboard')
plt.show()

## 18. Feature Importance Analysis - Permutation Importance

In [ ]:
# ============================================================
# 18. Permutation Importance for Both Projects
# ============================================================
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Sales - Permutation Importance (RF)
perm_imp_sales = permutation_importance(rf_best, X_test_scaled, y_test, n_repeats=10, 
                                         random_state=RANDOM_STATE, n_jobs=-1)
perm_sales_df = pd.Series(perm_imp_sales.importances_mean, index=feature_cols).sort_values(ascending=True)
perm_sales_df.tail(15).plot(kind='barh', ax=axes[0], color='steelblue', edgecolor='black')
axes[0].set_title('Sales RF: Permutation Importance (Top 15)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Mean Importance')

# Churn - Permutation Importance (XGBoost)
perm_imp_churn = permutation_importance(xgb_c_model, X_test_c_scaled, y_test_c, n_repeats=10,
                                         random_state=RANDOM_STATE, n_jobs=-1, scoring='f1')
perm_churn_df = pd.Series(perm_imp_churn.importances_mean, index=churn_feature_names).sort_values(ascending=True)
perm_churn_df.tail(15).plot(kind='barh', ax=axes[1], color='orange', edgecolor='black')
axes[1].set_title('Churn XGB: Permutation Importance (Top 15)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Mean Importance')

plt.tight_layout()
save_chart(fig, '12_permutation_importance.png', 'comparison')
plt.show()

## 19. Export Results & Final Chart Gallery

In [ ]:
# ============================================================
# 19. Export All Results to CSV
# ============================================================

# Save model comparison results
sales_comparison.to_csv('charts/sales_model_comparison.csv', index=False)
churn_comparison.to_csv('charts/churn_model_comparison.csv', index=False)

# Save sales predictions
sales_pred_df = pd.DataFrame({
    'Date': dates_test,
    'Actual_Sales': y_test,
    'RF_Predicted': rf_pred,
    'XGB_Predicted': xgb_pred
})
sales_pred_df.to_csv('charts/sales_predictions.csv', index=False)

# Save churn predictions
churn_pred_df = pd.DataFrame({
    'Actual_Churn': y_test_c.values,
    'LR_Predicted': lr_pred,
    'RF_Predicted': rf_c_pred,
    'XGB_Predicted': xgb_c_pred,
    'DNN_Predicted': dnn_pred,
    'LR_Probability': lr_prob,
    'RF_Probability': rf_c_prob,
    'XGB_Probability': xgb_c_prob,
    'DNN_Probability': dnn_prob
})
churn_pred_df.to_csv('charts/churn_predictions.csv', index=False)

# Save feature importances
pd.DataFrame({
    'Feature': feature_cols,
    'RF_Sales_Importance': rf_best.feature_importances_,
    'XGB_Sales_Importance': xgb_model.feature_importances_
}).to_csv('charts/sales_feature_importances.csv', index=False)

pd.DataFrame({
    'Feature': churn_feature_names,
    'RF_Churn_Importance': rf_c_best.feature_importances_,
    'XGB_Churn_Importance': xgb_c_model.feature_importances_
}).to_csv('charts/churn_feature_importances.csv', index=False)

print("✅ All results exported to CSV files!")

In [ ]:
# ============================================================
# Final Directory Listing & Summary
# ============================================================
print("=" * 80)
print("📁 COMPLETE CHART DIRECTORY LISTING")
print("=" * 80)

total_files = 0
total_size = 0
for root, dirs, files in os.walk(CHARTS_DIR):
    level = root.replace(CHARTS_DIR, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f'{indent}📁 {os.path.basename(root)}/')
    subindent = ' ' * 2 * (level + 1)
    for file in sorted(files):
        filepath = os.path.join(root, file)
        size = os.path.getsize(filepath)
        total_size += size
        total_files += 1
        print(f'{subindent}📄 {file} ({size/1024:.1f} KB)')

print(f"\n{'=' * 80}")
print(f"📊 Total charts saved: {total_files}")
print(f"💾 Total size: {total_size/1024/1024:.2f} MB")
print(f"{'=' * 80}")

print(f"""
{'=' * 80}
🏆 FINAL SUMMARY - AI-DRIVEN PREDICTIVE ANALYTICS
{'=' * 80}

📊 PROJECT 1: SALES FORECASTING
{'─' * 40}
• Dataset: {len(sales_df)} records, {sales_df['Order Date'].min().strftime('%Y-%m-%d')} to {sales_df['Order Date'].max().strftime('%Y-%m-%d')}
• Models Trained: Random Forest, XGBoost, LSTM
• Best Model: {sales_comparison.loc[sales_comparison['R²'].idxmax(), 'Model']}
  - R² Score: {sales_comparison['R²'].max():.4f}
  - MAE: ${sales_comparison.loc[sales_comparison['R²'].idxmax(), 'MAE']:.2f}
  - RMSE: ${sales_comparison.loc[sales_comparison['R²'].idxmax(), 'RMSE']:.2f}
• Key Drivers: Lag features, rolling averages, seasonal patterns

👥 PROJECT 2: CUSTOMER CHURN PREDICTION
{'─' * 40}
• Dataset: {len(churn_df)} customers
• Overall Churn Rate: {(churn_df['Churn'] == 'Yes').mean()*100:.1f}%
• Models Trained: Logistic Regression, Random Forest, XGBoost, DNN
• Best Model: {churn_comparison.loc[churn_comparison['AUC-ROC'].idxmax(), 'Model']}
  - AUC-ROC: {churn_comparison['AUC-ROC'].max():.4f}
  - F1-Score: {churn_comparison.loc[churn_comparison['AUC-ROC'].idxmax(), 'F1-Score']:.4f}
  - Recall: {churn_comparison.loc[churn_comparison['AUC-ROC'].idxmax(), 'Recall']:.4f}
• Key Churn Drivers: Contract type, tenure, monthly charges, internet service

💡 BUSINESS RECOMMENDATIONS
{'─' * 40}
1. Use the best sales model for monthly revenue forecasting
2. Implement churn prediction for proactive customer retention
3. Focus on Month-to-Month contract customers (highest churn risk)
4. Target high-risk customers with retention offers
5. Leverage seasonal patterns for inventory and staffing planning
6. Monitor feature importance changes over time for model retraining

{'=' * 80}
✅ Analysis Complete! All charts saved to 'charts/' directory.
{'=' * 80}
""")